## Creating an excel reports using remayn

### 1. Running some experiments with GridSearchCV and saving the results

A Logistic Regression model and a Ridge Classifier are trained using a GridSearch cross-validation procedure. Then, the results are saved including the best parameters found.

In [1]:
from remayn.report import create_excel_summary_report, create_excel_columns_report
from remayn.result import make_result
from remayn.result_set import ResultFolder
from shutil import rmtree
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import time

In [2]:

# Clean up the results folder if exists
rmtree('./results', ignore_errors=True)

# Repeat the experiment 10 times with different random seeds
for seed in range(10):
    for model, param_grid in [(LogisticRegression, {'C': [0.1, 1, 10], 'max_iter': [50, 100, 150]}),
                              (RidgeClassifier, {'alpha': [0.1, 1, 10], 'max_iter': [50, 100, 150]})]:
        # Generate a sample dataset
        X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=2, n_clusters_per_class=2, random_state=0)

        # Split the dataset into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        # Train the model
        start_time = time.time()

        gs = GridSearchCV(model(), param_grid=param_grid, cv=5)
        gs.fit(X_train, y_train)

        train_time = time.time() - start_time

        # Make predictions
        y_train_pred = gs.predict(X_train)
        y_test_pred = gs.predict(X_test)

        # Prepare estimator config that is going to be saved
        estimator_config = gs.get_params()
        # Remove the 'estimator' key from the config, as it is not serializable
        estimator_config.pop('estimator')

        # Create a dictionary that represents the config of this experiment.
        # Any information relevant for the experiment can be included here.
        # In this case, all the hyperparameters of the estimator are included.
        experiment_config = {
            "estimator_config": estimator_config,
            "estimator_name": model.__name__,
            "seed": seed,
            "dataset": "example",
        }

        # Save the results of the experiment
        make_result(
            base_path='./results',
            config=experiment_config,
            targets=y_test,
            predictions=y_test_pred,
            train_targets=y_train,
            train_predictions=y_train_pred,
            time=train_time,

            # Save the best hyperparameters and the best model
            best_params=gs.best_params_,
            best_model=gs.best_estimator_
        ).save()



### 2. Create a report using these results

First, we load the results folder:

In [3]:
# Load the results from the folder
rf = ResultFolder('./results')
rf

ResultSet with 20 results

Then, we create the dataframe with the desired metrics:

In [4]:
# Create the dataframe with the results
def compute_metrics(targets, predictions):
    return {
        "accuracy": accuracy_score(targets, predictions),
        "precision": precision_score(targets, predictions),
        "recall": recall_score(targets, predictions),
        "f1": f1_score(targets, predictions),
    }

config_colums = [
    "dataset",
    "estimator_name",
    "seed",

    # Nested values can be accessed using dot notation
    "estimator_config.param_grid.C",
    "estimator_config.param_grid.max_iter",
]

df = rf.create_dataframe(
    config_columns=config_colums,
    metrics_fn=compute_metrics,
    # By default only the test metrics are included
    include_train=True,
    # By default, the prefix is config_
    config_columns_prefix=""
)

df

,dataset,estimator_name,seed,estimator_config.param_grid.C,estimator_config.param_grid.max_iter,accuracy,precision,recall,f1,train_accuracy,train_precision,train_recall,train_f1,time
0,example,LogisticRegression,2,"[0.1, 1, 10]","[50, 100, 150]",0.950,0.963855,0.919540,0.941176,0.94875,0.957921,0.941606,0.949693,0.402048
1,example,LogisticRegression,4,"[0.1, 1, 10]","[50, 100, 150]",0.930,0.926316,0.926316,0.926316,0.95500,0.959900,0.950372,0.955112,0.385756
2,example,RidgeClassifier,3,None,"[50, 100, 150]",0.965,0.970874,0.961538,0.966184,0.94000,0.943590,0.934010,0.938776,0.265680
3,example,LogisticRegression,0,"[0.1, 1, 10]","[50, 100, 150]",0.960,0.962617,0.962617,0.962617,0.94250,0.950392,0.930946,0.940568,0.405789
4,example,LogisticRegression,9,"[0.1, 1, 10]","[50, 100, 150]",0.930,0.935185,0.935185,0.935185,0.95875,0.963636,0.951282,0.957419,0.406158
5,example,RidgeClassifier,5,None,"[50, 100, 150]",0.940,0.958763,0.920792,0.939394,0.94500,0.946835,0.942065,0.944444,0.309319
6,example,RidgeClassifier,0,None,"[50, 100, 150]",0.960,0.962617,0.962617,0.962617,0.94125,0.945596,0.933504,0.939511,0.417669
7,example,RidgeClassifier,9,None,"[50, 100, 150]",0.920,0.933962,0.916667,0.925234,0.95000,0.953368,0.943590,0.948454,0.268390
8,example,LogisticRegression,6,"[0.1, 1, 10]","[50, 100, 150]",0.950,0.938776,0.958333,0.948454,0.95125,0.957179,0.945274,0.951189,0.395839
9,example,RidgeClassifier,1,None,"[50, 100, 150]",0.935,0.971963,0.912281,0.941176,0.94125,0.944591,0.932292,0.938401,0.342123


Finally, we create the excel reports:

In [5]:
# Columns that will be used to group the results and compute means
groups_columns = ["dataset", "estimator_name"]

# Metrics columns that will be used to create the columns report
# This report can be useful to compute statistical tests as it shows one methodology
# per column and all the results (seeds) for that methodology in the rows
# It will create one sheet per metric
metric_columns = ['accuracy', 'precision', 'recall', 'f1']

create_excel_summary_report(df, 'summary_report.xlsx', group_columns=groups_columns)
create_excel_columns_report(df,
                            'columns_report.xlsx',
                            metric_columns=metric_columns,
                            pivot_index="seed",
                            pivot_columns=groups_columns)

PosixPath('columns_report.xlsx')

We can also create a single excel containing all the sheets, using the `pandas.ExcelWriter`:

In [6]:
# Columns that will be used to group the results and compute means
groups_columns = ["dataset", "estimator_name"]

# Metrics columns that will be used to create the columns report
# This report can be useful to compute statistical tests as it shows one methodology
# per column and all the results (seeds) for that methodology in the rows
# It will create one sheet per metric
metric_columns = ['accuracy', 'precision', 'recall', 'f1']


with pd.ExcelWriter('report.xlsx') as writer:
    create_excel_summary_report(df, 'summary_report.xlsx',
                                group_columns=groups_columns,
                                excel_writer=writer)
    create_excel_columns_report(df,
                                'columns_report.xlsx',
                                metric_columns=metric_columns,
                                pivot_index="seed",
                                pivot_columns=groups_columns,
                                excel_writer=writer)